# Screening of functionalized hemithioindigos

## Starting with a small family of 6 molecules we will be performing the following calculations in order to determine how properties change with respect to the parent hemithioindigo photoswitch

### A small note on what this notebook contains:
This currently uses Python 2.7 which I have chosen due to it being the only Python build which I have:
a) a working version of obcalc
b) a working implementation of ORCA which can be hypervised using ASE
I may not do every calculation in ASE, but certainly will be using it to generate inputs for ORCA calculations that will be run straight from the shell.

## Part 1: Generating isomer geometries

In [16]:
from ase.atoms import Atoms, Atom
from ase.constraints import FixInternals, FixAtoms
from ase.visualize import view
from ase.optimize import QuasiNewton, BFGS
from math import pi
from ase.io import write,read
import obcalc
import os, shutil
import numpy as np

This next cell generates opposite isomers

In [28]:
for i in range(1,7):
    mol = read('docs/geometries/'+str(i)+'-cis.xyz')
    d_indices = [12, 7, 14, 15]
    if len(mol.get_atomic_numbers()) == 27:
        mask = np.zeros(14).tolist() + np.ones(13).tolist()
    if len(mol.get_atomic_numbers()) > 27:
        mask = np.zeros(14).tolist() + np.ones(13).tolist() + np.zeros(len(mol.get_atomic_numbers())-27).tolist()
    if len(mol.get_atomic_numbers()) == 34:
        mask = np.zeros(14).tolist() + np.ones(13).tolist() + np.zeros(len(mol.get_atomic_numbers())-31).tolist() + np.ones(4).tolist()
    mol.set_dihedral(d_indices, pi, mask)
    dmasses=[mol.get_masses()[d_indices[0]],mol.get_masses()[d_indices[1]],mol.get_masses()[d_indices[2]],mol.get_masses()[d_indices[3]]]
    dihedral_constraint = [mol.get_dihedral(d_indices), d_indices, dmasses]
    constraint = [FixInternals(dihedrals=[dihedral_constraint]), FixAtoms(indices=[12])]
    mol.set_constraint(constraint)
    mol.set_calculator(obcalc.OBForceField('uff', bonds=obcalc.get_bonds(mol)))
    relax = BFGS(mol)
    relax.run(fmax=0.1)
    mol.write('docs/geometries/'+str(i)+'-trans.xyz')

      Step     Time          Energy         fmax
BFGS:    0 10:03:59      219.020208     2128.7762
BFGS:    1 10:03:59      103.904208      933.7679
BFGS:    2 10:03:59       82.925905      731.5921
BFGS:    3 10:03:59       74.009947      644.7221
BFGS:    4 10:03:59       47.185700      384.1765
BFGS:    5 10:03:59       34.954732      275.1271
BFGS:    6 10:03:59       24.403384      177.7842
BFGS:    7 10:03:59       16.515771      110.7695
BFGS:    8 10:03:59       11.685522       70.1166
BFGS:    9 10:03:59        8.470396       43.3566
BFGS:   10 10:03:59        6.391648       26.6004
BFGS:   11 10:03:59        5.160878       16.7357
BFGS:   12 10:03:59        4.426527       10.8202
BFGS:   13 10:03:59        3.954529        6.5706
BFGS:   14 10:03:59        3.650937        3.9704
BFGS:   15 10:03:59        3.454757        1.9604
BFGS:   16 10:03:59        3.328725        0.6495
BFGS:   17 10:03:59        3.249576        0.5106
BFGS:   18 10:03:59        3.200195        1.1240
B

This below is just a little function to get rid of the ASE tags that it writes to the .xyz file. ORCA finds issues with some of them, particularly the ones that state periodic boundary conditions (PBC), so it is just better to remove them. Using the script below, using only the filename, it will comment out the 2nd line of the geometry file which contains these.

In [32]:
def remove_geomtags(filename):
    with open(filename, 'r') as file:
        list_of_lines = file.readlines()
        list_of_lines[1] = '# \n'
        file.close()
    with open(filename, 'w') as file:
        file.writelines(list_of_lines)
        file.close()   
        print(filename+' succesfully edited!')     

In [33]:
mol_id = ["%1d" % i for i in range(1,7)]; mol_state = ['cis', 'trans']
for i in mol_id:
    for j in mol_state:
        remove_geomtags('docs/geometries/'+i+'-'+j+'.xyz')

docs/geometries/1-cis.xyz succesfully edited!
docs/geometries/1-trans.xyz succesfully edited!
docs/geometries/2-cis.xyz succesfully edited!
docs/geometries/2-trans.xyz succesfully edited!
docs/geometries/3-cis.xyz succesfully edited!
docs/geometries/3-trans.xyz succesfully edited!
docs/geometries/4-cis.xyz succesfully edited!
docs/geometries/4-trans.xyz succesfully edited!
docs/geometries/5-cis.xyz succesfully edited!
docs/geometries/5-trans.xyz succesfully edited!
docs/geometries/6-cis.xyz succesfully edited!
docs/geometries/6-trans.xyz succesfully edited!


 The created geometries are optimized using a forcefield, however we also want to optimize the created geometry using our QM-method DFT. I am using B3LYP, with the D3BJ correction, with a def2-TZVP basis set. I don't want to actually run these calculations just yet at the moment, so I am just saving this as a python script to launch on my HPC of choice. Again, I have chosen for the S atom to stay static. This is to ensure only the hemistilbene section of the ring moves for now. It's also a good reference point.

In [36]:
%%writefile temp/qm_globalopt.py 
import os,shutil
for i in ["%1d" % i for i in range(1,7)]:
    os.mkdir(i)
    for j in ['cis', 'trans']:    
        os.mkdir(i+'/'+j)
        shutil.copy('geometries/'+i+'-'+j+'.xyz', i+'/'+j+'/startgeom.xyz')
        inp = open(i+'/'+j+'/input.inp', 'w+')
        inp.write('! B3LYP D3BJ def2-TZVP RIJCOSX \n'
                  '! Opt \n'
                  '%pal nprocs 48 end \n'
                  '%geom Constraints { C 12 C } end end \n'
                  '%base "optimized" \n'
                  '* xyzfile 0 1 startgeom.xyz \n')
        inp.close()
        os.chdir(i+'/'+j)
        os.system('python ~/subgen-2.py '+i+j+'opt 1 orca501 4 m.lea')
        os.system('sbatch submit.sh')
        os.chdir('../..')

Overwriting temp/qm_globalopt.py


## Locating a minimum energy path using nudged elastic band (NEB)
I plan on using 11 images for my ORCA neb (this doesn't include the first or last image) because that would result in 15deg intervals for dihedral angles, assuming that is the path it will default to. It of course won't, but if we compare NEBs in aims using predetermined dihedral paths vs interpolation (IDPP) it certainly would help to have the same number of images. I am planning on performing OptTS calculations on all highest energy images in these NEB calculations.

In [38]:
%%writefile temp/orca_idpp_neb.py 
import os,shutil
for i in ["%1d" % i for i in range(1,7)]:
    os.mkdir(i+'/NEB')
    os.chdir(i+'/NEB')
    shutil.copy(i+'/trans/optimized.xyz', './start.xyz')   
    shutil.copy(i+'/cis/optimized.xyz', './end.xyz')
    inp = open(i+'/NEB/input.inp', 'w+')
    inp.write('! B3LYP D3BJ def2-TZVP RIJCOSX \n'
              '! NEB \n'
              '%pal nprocs 48 end \n'
              '%base "neb" \n'
              '%neb NEB_END_XYZFile "end.xyz" \n'
              'NImages 11 \n'
              'Monitor_Internals { D 12 7 14 15 } end \n'
              'end \n'
              '* xyzfile 0 1 start.xyz')
    inp.close()
    os.system('python ~/subgen-2.py '+i+'-neb 1 orca501 48 m.lea')
    os.system('sbatch submit.sh')
    os.chdir('../..')

Overwriting temp/orca_idpp_neb.py


After the optimizations, I can perform an eigenvector following optimization using the OptTS keyword.

In [2]:
%%writefile temp/orca_optts.py
import os,shutil
for i in ["%1d" % i for i in range(1,7)]:
    os.mkdir(i+'/NEB/ts-opt/')
    os.chdir(i+'/NEB/ts-opt/')
    shutil.copy('../neb_NEB-HEI_converged.xyz', './ts.xyz')   
    inp = open('./input.inp', 'w+')
    inp.write('! B3LYP D3BJ def2-TZVP RIJCOSX \n'
              '! OptTS \n'
              '%pal nprocs 48 end \n'
              '%base "tsopt" \n'
              '* xyzfile 0 1 ts.xyz \n')
    inp.close()
    os.system('python ~/subgen-2.py '+i+'-ts 1 orca501 48 m.lea')
    os.system('sbatch submit.sh')
    os.chdir('../../../')

Writing temp/orca_optts.py


## Vibrational spectra on NEB images
### Once we have a converged NEB calculation, we should go and optimize the highest energy image using eigenvector following methods, (OptTS). Whilst we do this, we could also try and get the vibrational spectra of each image in order to do a pseudo-simulation of time-resolved vibrational spectra (TVAS). This could also perhaps be done in the excited state.